In [3]:
from apify_client import ApifyClient
from dotenv import load_dotenv
load_dotenv()
import os


# Initialize the ApifyClient with your API token
# client = ApifyClient(os.getenv('APIFY_kEY'))

# # Prepare the Actor input
# run_input = {
#     "queries": """javascript
#                 typescript
#                 python""",
#     "resultsPerPage": 100,
#     "maxPagesPerQuery": 1,
#     "languageCode": "",
#     "forceExactMatch": False,
#     "wordsInTitle": [],
#     "wordsInText": [],
#     "wordsInUrl": [],
#     "mobileResults": False,
#     "includeUnfilteredResults": False,
#     "saveHtml": False,
#     "saveHtmlToKeyValueStore": True,
#     "includeIcons": False,
# }

# # Run the Actor and wait for it to finish
# run = client.actor("nFJndFXA5zjCTuudP").call(run_input=run_input)

# # Fetch and print Actor results from the run's dataset (if there are any)
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    # print(item)

In [14]:
# from apify_client import ApifyClient

# # You can find your API token at https://console.apify.com/settings/integrations.
# TOKEN = 'MY-APIFY-TOKEN'


# def main() -> None:
#     apify_client = ApifyClient(TOKEN)

#     # Start an Actor and wait for it to finish.
#     actor_client = apify_client.actor('john-doe/my-cool-actor')
#     call_result = actor_client.call()

#     if call_result is None:
#         print('Actor run failed.')
#         return

#     # Fetch results from the Actor run's default dataset.
#     dataset_client = apify_client.dataset(call_result['defaultDatasetId'])
#     list_items_result = dataset_client.list_items()
#     print(f'Dataset: {list_items_result}')




from apify_client import ApifyClientAsync
import pandas as pd
# You can find your API token at https://console.apify.com/settings/integrations.
TOKEN = os.getenv('APIFY_KEY')
# Prepare the Actor input
run_input = {
    "queries": """real estate
                constructions
                housing properties""",
    "resultsPerPage": 2,
    "maxPagesPerQuery": 1,
    "languageCode": "de",
    "forceExactMatch": False,
    "wordsInTitle": [],
    "wordsInText": [],
    "wordsInUrl": [],
    "mobileResults": False,
    "includeUnfilteredResults": False,
    "saveHtml": False,
    "saveHtmlToKeyValueStore": True,
    "includeIcons": False,
}

async def main() -> None:
    apify_client = ApifyClientAsync(TOKEN)

    # Start an Actor and wait for it to finish.
    actor_client = apify_client.actor('apify/google-search-scraper')
    call_result = await actor_client.call(run_input=run_input)

    if call_result is None:
        print('Actor run failed.')
        return

    # Fetch results from the Actor run's default dataset.
    dataset_client = apify_client.dataset(call_result['defaultDatasetId'])
    list_items_result = await dataset_client.list_items()
    print(f'Dataset: {pd.DataFrame(list_items_result.items)}')
    

await main()


Dataset:                                               #debug  #error  \
0  {'requestId': 'jOJi5Yl6FoNLXH4', 'url': 'http:...   False   
1  {'requestId': 'XvD3XiAusaTNex9', 'url': 'http:...   False   
2  {'requestId': '3zuQsbtbqbKrFZh', 'url': 'http:...   False   

                                         searchQuery  \
0  {'term': 'real estate', 'url': 'http://www.goo...   
1  {'term': 'housing properties', 'url': 'http://...   
2  {'term': 'constructions', 'url': 'http://www.g...   

                                                 url  hasNextPage  \
0  http://www.google.com/search?q=real+estate&hl=...         True   
1  http://www.google.com/search?q=housing+propert...         True   
2  http://www.google.com/search?q=constructions&h...         True   

  serpProviderCode  resultsTotal  \
0                L    6220000000   
1                L     846000000   
2                L    3090000000   

                                      relatedQueries paidResults paidProducts  \
0  [{'

In [ ]:
from apify_client import ApifyClientAsync
import asyncio
import aiohttp
import logging
from typing import List, Dict, Optional
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

class LeadGenerator:
    def __init__(self, token: str):
        self.client = ApifyClientAsync(token)
        self.logger = logging.getLogger(__name__)
        self.apify_token = os.getenv('APIFY_TOKEN')
        self.email_config = {
            'sender': os.getenv('SENDER_EMAIL'),
            'password': os.getenv('EMAIL_PASSWORD'),
            'smtp_server': 'smtp.gmail.com',
            'port': 465
        }
        
            
    async def fetch_search_results(self, keywords: List[str]) -> Optional[pd.DataFrame]:
        query = "\n".join(keywords)
        run_input = {
            "queries": query,
            "resultsPerPage": 2,
            "maxPagesPerQuery": 1,
            "languageCode": "de",
            "countryCode": 'de',
            # ... rest of config
        }
        try:
            actor_client = self.client.actor('apify/google-search-scraper')
            call_result = await actor_client.call(run_input=run_input)
            if not call_result:
                raise ValueError('Search actor run failed')
                
            dataset_client = self.client.dataset(call_result['defaultDatasetId'])
            async for item in dataset_client.iterate_items():
                for result in item.get('organicResults', []):
                    yield {
                        'title': result.get('title', ''),
                        'link': result.get('url', ''),
                        'description': result.get('description', ''),
                    }
        except Exception as e:
            self.logger.error(f"Error fetching search results: {str(e)}")
            raise


class ContactGenerator:
    def __init__(self, token: str):
        self.client = ApifyClientAsync(token)
        self.logger = logging.getLogger(__name__)
            
    async def fetch_contact_results(self, url: str) -> Optional[pd.DataFrame]:
        # query = "\n".join(keywords)
        run_input = {
            "startUrls": [{ "url": url }],
            "maxRequestsPerStartUrl": 1,
            "maxDepth": 5,
            "maxRequests": 9999999,
            "sameDomain": True,
            "considerChildFrames": True,
            "useBrowser": False,
            "waitUntil": "domcontentloaded",
            "proxyConfig": { "useApifyProxy": True },
        }

        try:
            actor_client = self.client.actor('vdrmota/contact-info-scraper')
            call_result = await actor_client.call(run_input=run_input)
            if not call_result:
                raise ValueError('Search actor run failed')
                
            dataset_client = self.client.dataset(call_result['defaultDatasetId'])
            async for item in dataset_client.iterate_items():
                emails = item.get('emails', [])
                if emails != []:
                    yield {'email': emails[0]}
                else:
                    yield {'email': None}
        except Exception as e:
            self.logger.error(f"Error fetching contact results: {str(e)}")
            raise

async def main():
    """Main function to demonstrate usage."""
    # Initialize generators
    token = os.getenv('APIFY_KEY')
    lead_gen = LeadGenerator(token)
    contact_gen = ContactGenerator(token)


    try:
        # Process leads sequentially
        async for lead_result in lead_gen.fetch_search_results(['real estate companies', 'construction companies', 'realtors', 'real estate agents', 'interior designers and decor', 'property developers']):
            try:
                # Fetch contact information for each lead
                contacts = [contact async for contact in contact_gen.fetch_contact_results(lead_result.get('link', ''))]
                
                if contacts and contacts[0].get('email'):
                    lead_result.update(contacts[0])
                else:
                    continue


                print(lead_result)
            except Exception as e:
                contact_gen.logger.error(
                    f"Error processing {lead_result['link']}: {str(e)}"
                )
                continue
        return lead_result

    except Exception as e:
        contact_gen.logger.error(f"Main function error: {str(e)}")
        raise

if __name__ == "__main__":
    asyncio.run(main())

{'title': 'LEONHARD WEISS Construction Company', 'link': 'https://www.leonhard-weiss.de/eng/index_en.html', 'description': "Founded in 1900, LEONHARD WEISS is among Germany's most powerful and successful construction companies.", 'email': 'bau-de@leonhard-weiss.com'}
{'title': 'What is a property developer, what do they do & ...', 'link': 'https://www.mfsuk.com/blog/what-is-a-property-developer/', 'description': 'Property developers are individuals who build new properties or refurbish existing houses to sell them on for a profit.', 'email': 'info@mfsuk.com'}


In [38]:
class LeadGenerator:
    def __init__(self, token: str):
        self.client = ApifyClientAsync(token)
        self.logger = logging.getLogger(__name__)

    async def fetch_search_results(self, keywords: List[str]) -> List[Dict]:
        query = "\n".join(keywords)
        run_input = {
            "queries": query,
            "resultsPerPage": 2,
            "maxPagesPerQuery": 1,
            "languageCode": "de",
            "countryCode": 'de',
        }
        try:
            actor_client = self.client.actor('apify/google-search-scraper')
            call_result = await actor_client.call(run_input=run_input)
            
            if not call_result:
                raise ValueError('Search actor run failed')
                
            dataset_client = self.client.dataset(call_result['defaultDatasetId'])
            results = []
            async for item in dataset_client.iterate_items():
                for result in item.get('organicResults', []):
                    results.append({
                        'title': result.get('title', ''),
                        'link': result.get('url', ''),
                        'description': result.get('description', ''),
                    })
            return results
        except Exception as e:
            self.logger.error(f"Error fetching search results: {str(e)}")
            raise

class ContactGenerator:
    def __init__(self, token: str):
        self.client = ApifyClientAsync(token)
        self.logger = logging.getLogger(__name__)

    async def fetch_contact_results(self, url: str) -> Optional[pd.DataFrame]:
        run_input = {
            "startUrls": [{"url": url}],
            "maxRequestsPerStartUrl": 1,
            "maxDepth": 2,
            "maxRequests": 9999999,
            "sameDomain": True,
            "considerChildFrames": True,
            "useBrowser": False,
            "waitUntil": "domcontentloaded",
            "proxyConfig": {"useApifyProxy": True},
        }
        try:
            actor_client = self.client.actor('vdrmota/contact-info-scraper')
            call_result = await actor_client.call(run_input=run_input)
            
            if not call_result:
                raise ValueError('Search actor run failed')
                
            dataset_client = self.client.dataset(call_result['defaultDatasetId'])
            items_list = await dataset_client.list_items()
            items_data = [item.get('data') for item in items_list.items]
            df = pd.DataFrame(items_data) if items_data else None
            return df if df is not None and not df.empty else None
        except Exception as e:
            self.logger.error(f"Error fetching contact results: {str(e)}")
            raise

async def main():
    token = os.getenv('APIFY_KEY')
    lead_gen = LeadGenerator(token)
    contact_gen = ContactGenerator(token)
    
    try:
        leads = await lead_gen.fetch_search_results(['real estate'])
        results_table = []
        for lead_result in leads:
            try:
                contact_df = await contact_gen.fetch_contact_results(lead_result['link'])
                if contact_df is not None and 'emails' in contact_df.columns:
                    results_table.append(contact_df['emails'][0])
                    print(results_table)
            except Exception as e:
                contact_gen.logger.error(
                    f"Error processing {lead_result['link']}: {str(e)}"
                )
        return pd.DataFrame(results_table, columns=['email']) if results_table else None
    except Exception as e:
        contact_gen.logger.error(f"Main function error: {str(e)}")
        raise

In [39]:
result = await main()
result

In [ ]:
import asyncio
from apify_client import ApifyClientAsync
import ollama
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
from typing import List, Dict, Optional
import logging
from datetime import datetime
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
load_dotenv()

class LeadPipeline:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.apify_token = os.getenv('APIFY_TOKEN')
        self.email_config = {
            'sender': os.getenv('SENDER_EMAIL'),
            'password': os.getenv('EMAIL_PASSWORD'),
            'smtp_server': 'smtp.gmail.com',
            'port': 465
        }
        
    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
    
    async def fetch_leads(self, keywords: List[str]) -> Optional[pd.DataFrame]:
        client = ApifyClientAsync(self.apify_token)
        scraper = LeadScraper(client)
        try:
            leads_df = await scraper.get_leads(keywords)
            return leads_df
        except Exception as e:
            self.logger.error(f"Error fetching leads: {str(e)}")
            return None

    def generate_email_content(self, prospect_details: str) -> Dict[str, str]:
        template = f"""
        You are a professional prospector reaching out to a potential client. Using the details below, 
        generate a personalized email outreach message in German.
        PROSPECTOR DETAILS:
        -------------------
        Website: Expose´profi.de
        Company Name: Expose´Profi
        Description: The requirements of the real estate market and the demands of customers are constantly changing. 
                    As a property developer or real estate agent, you must always have the right solutions ready to successfully meet both sides' needs.
                    Abstract drawings, floor plans, and sketches are no longer sufficient to present a project attractively. 
                    Prospective buyers want to see an emotional representation and receive comprehensive advice about their visions. 
                    As an innovation leader in 3D visualization of real estate, we accompany you during the planning phase of your new building project.
                    We help you present your brand successfully and stand out from the competition with high-quality 3D visualizations.
                    We are both innovative and experienced, following developments in real estate marketing closely. 
                    In our internal processes, we rely on the latest technologies in 3D modeling. Tasteful visualizations are perfect for your sales in online and offline marketing: on real estate websites, in exposés, printed materials, social media, etc.
        PROSPECT DETAILS:
        -----------------
        {prospect_details}
        EMAIL REQUIREMENTS:
        -------------------
        - Tone: Professional and friendly.
        - Content: Concise and to the point.
        - Must clearly state how our service benefits their project or business.
        - Ending: Include a polite sign-off with your contact details.
        OUTPUT FORMAT:
        --------------
        Return a JSON object with exactly two keys: "Subject" and "Body". Their values must be strings.
        Do not include any extra text or commentary.
        Generate the email message in German with a complete subject line and body using the details provided.
        Output: {{
            "Subject": "subject of the email",
            "Body": "body of the email"
        }}
        """
        try:
            response = ollama.generate(model='llama3.2:latest', prompt=template)
            result = response['response']
            return {
                'subject': result['Subject'],
                'body': result['Body']
            }
        except Exception as e:
            self.logger.error(f"Error generating email content: {str(e)}")
            return None

    def send_email(self, recipient: str, subject: str, body: str) -> bool:
        if not self.email_config['sender'] or not self.email_config['password']:
            self.logger.error("Email credentials missing")
            return False
            
        msg = MIMEMultipart()
        msg['From'] = self.email_config['sender']
        msg['To'] = recipient
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))
        
        try:
            with smtplib.SMTP_SSL(
                self.email_config['smtp_server'],
                self.email_config['port']
            ) as server:
                server.login(
                    self.email_config['sender'],
                    self.email_config['password']
                )
                server.send_message(msg)
            return True
        except Exception as e:
            self.logger.error(f"Error sending email: {str(e)}")
            return False

    async def process_leads(self, keywords: List[str]) -> int:
        """Main pipeline function that orchestrates the entire process"""
        leads_df = await self.fetch_leads(keywords)
        if leads_df is None:
            self.logger.error("Failed to fetch leads")
            return 0
            
        sent_count = 0
        for _, lead in leads_df.iterrows():
            prospect_details = f"""
            Company Name: {lead.get('title', '')}
            Website: {lead.get('link', '')}
            Description: {lead.get('description', '')}
            """
            
            # Generate email content
            email_data = self.generate_email_content(prospect_details)
            if not email_data:
                continue
                
            # Send email
            success = self.send_email(
                lead.get('email', ''),
                email_data['subject'],
                email_data['body']
            )
            if success:
                sent_count += 1
                
        return sent_count

async def main():
    pipeline = LeadPipeline()
    pipeline.setup_logging()
    
    keywords = ['real estate agents', 'construction companies']
    sent_emails = await pipeline.process_leads(keywords)
    print(f"Successfully sent emails to {sent_emails} recipients")

if __name__ == "__main__":
    asyncio.run(main())